In [18]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm
import category_encoders as ce


# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss
from sklearn.impute import KNNImputer

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

In [19]:
train_set = pd.read_csv('train_prepared.csv', encoding='utf-8')
test_set = pd.read_csv('test_prepared.csv', encoding='utf-8')

train_set = train_set.replace([-np.inf, np.inf], np.nan)
test_set = test_set.replace([-np.inf, np.inf], np.nan)

In [20]:
train_fts = train_set.columns

imputer_train = KNNImputer()
imputer_train.fit(train_set)
train_set = pd.DataFrame(imputer_train.transform(train_set))

train_set.columns = train_fts

In [21]:
X = train_set.drop(columns=['Target', 'Opportunity_ID'])
y = train_set.Target

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [23]:
'''
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 2000, num = 5)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
'''

"\nn_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]\n\nmax_features = ['auto', 'sqrt']\n\nmax_depth = [int(x) for x in np.linspace(10, 110, num = 5)]\nmax_depth.append(None)\n\nmin_samples_split = [2, 5, 10]\n\nmin_samples_leaf = [1, 2, 4]\n\nbootstrap = [True, False]\n\nrandom_grid = {'n_estimators': n_estimators,\n               'max_features': max_features,\n               'max_depth': max_depth,\n               'min_samples_split': min_samples_split,\n               'min_samples_leaf': min_samples_leaf,\n               'bootstrap': bootstrap}\nprint(random_grid)\n"

In [24]:
'''
clf=RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train,y_train)

y_pred=rf_random.predict_proba(X_test)
'''

'\nclf=RandomForestClassifier()\nrf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)\n\nrf_random.fit(X_train,y_train)\n\ny_pred=rf_random.predict_proba(X_test)\n'

In [25]:
#rf_random.best_params_

In [26]:
#print(log_loss(y_test, y_pred))

In [39]:
c = RandomForestClassifier(n_estimators= 50, min_samples_split= 2, 
                           min_samples_leaf=5, max_features= 'sqrt', 
                           max_depth= 25,bootstrap= False)
c.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=25, max_features='sqrt',
                       min_samples_leaf=5, n_estimators=50)

In [40]:
from sklearn.metrics import f1_score

print(log_loss(y_test, c.predict_proba(X_test)))
f1_score(y_test, c.predict(X_test))

0.3484868895316722


0.8559077809798271

In [41]:
print(log_loss(y_train, c.predict_proba(X_train)))
f1_score(y_train, c.predict(X_train))

0.13769448506773432


0.9865348980852379

# EN TEST_SET

In [42]:
X_test_set = test_set.drop(columns=['Target', 'Opportunity_ID'])
y_test_set = test_set.Target

X_test_fts = X_test_set.columns

imputer_test = KNNImputer()
imputer_test.fit(X_test_set)
X_test_set = pd.DataFrame(imputer_test.transform(X_test_set))

X_test_set.columns=X_test_fts

In [43]:
test_pred = c.predict_proba(X_test_set)

In [55]:
test_set['Prediction'] = pd.DataFrame(test_pred)[1].to_list()
test_set.loc[:, ['Opportunity_ID', 'Prediction']]

,Opportunity_ID,Prediction
0,10689,0.738333
1,10690,0.579857
2,10691,0.465730
3,10692,0.354175
4,10693,0.922683
...,...,...
1562,12364,0.954349
1563,12365,0.367373
1564,12366,0.373444
1565,12367,0.528937


In [56]:
test_set['Prediction_Aux'] = 1 - test_set.Prediction
test_set.loc[:, ['Opportunity_ID', 'Prediction', 'Prediction_Aux']]

,Opportunity_ID,Prediction,Prediction_Aux
0,10689,0.738333,0.261667
1,10690,0.579857,0.420143
2,10691,0.465730,0.534270
3,10692,0.354175,0.645825
4,10693,0.922683,0.077317
...,...,...,...
1562,12364,0.954349,0.045651
1563,12365,0.367373,0.632627
1564,12366,0.373444,0.626556
1565,12367,0.528937,0.471063


In [57]:
log_loss(test_set.Target, test_set.Prediction_Aux)

1.7906476270138945

In [58]:
submission = test_set.loc[:, ['Opportunity_ID', 'Prediction']]
submission.columns = ['Opportunity_ID', 'Target']

In [59]:
submission.head()

,Opportunity_ID,Target
0,10689,0.738333
1,10690,0.579857
2,10691,0.465730
3,10692,0.354175
4,10693,0.922683


In [60]:
submission.to_csv('predictions/sub_rf.csv', index=False)